In [1]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
import torch.optim as optim
import yaml
import pickle
from torch.utils.data import Dataset, DataLoader

Carico il configuratore che tiene traccia di tutte le informazioni generali 

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
with open("./config.yaml", 'r') as f:
    config = yaml.safe_load(f)

In [3]:
config['dataset']['col_region']

['data',
 'codice_regione',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'dimessi_guariti',
 'deceduti']

# Download Dati

In questa parte carico i `.csv` e gli organizzo in modo tale che siano ordinati per

`(data, denominazione_regionale, denominazione_provincia)`

Inoltre pulisco i vari datase poiché ci sono una serie di righe che presentano delle problematicità in termini di valori.

La matrice di adiacenza generale è stata creata connettendo tutte le provincie di una determinata regione, in modo tale da mantenere una similitudine con il problema originario. Inoltre, per non distaccarsi troppo con la realtà, sono state inserite delle connessioni tra regioni dato che quest'ultime non sono isolate. In particolare sono state aggiunte connessioni tra tutti i capoluoghi.

In [4]:
def get_adj(config: yaml):
    if os.path.exists(os.path.join(config['paths']['adj'],"adj_totale.pkl")):
        with open(os.path.join(config['paths']['adj'],"adj_totale.pkl"), "rb") as f:
            adj = pickle.load(f)
    else:
        codice_reg_prov = pd.read_csv(os.path.join(config['paths']['data'], "codice_reg_prov.csv"), index_col=0)

        # Carico un dizionario in cui le chiavi sono le regioni e i valori sono i rispettivi capoluohi
        with open(os.path.join(config['paths']['data'], "capoluoghi.json"), 'r') as f:
            capoluoghi = json.load(f)
        
        adjs = {}
        adj = np.zeros((len(codice_reg_prov), len(codice_reg_prov)))
        i = 0
        index_capoluoghi = []
        d = codice_reg_prov.groupby('denominazione_provincia').codice_provincia.max().to_dict()
        regioni = codice_reg_prov.groupby(by=['codice_regione']).codice_provincia.unique()
        
        for cod_reg in tqdm(regioni.index):
            codici_provincia = regioni[cod_reg]
            n_prov = len(codici_provincia)
            adjs[cod_reg] = np.ones((n_prov, n_prov))-np.eye(n_prov)
            adj[i:i+n_prov, i:i+n_prov] = adjs[cod_reg]

            region = codice_reg_prov[codice_reg_prov.codice_regione==cod_reg].denominazione_regione.values[0]
            codice_capoluogo = codice_reg_prov[codice_reg_prov.denominazione_provincia==capoluoghi[region]].codice_provincia.values[0]
            pos = np.where(codici_provincia == codice_capoluogo)[0][0]
            index_capoluoghi.append(i+pos)
            
            
            with open(os.path.join(config['paths']['adj'],f"adj_{region}.pkl"), "wb") as f:
                pickle.dump(adj, f)
                
            i += n_prov
        for ind_r in index_capoluoghi:
            for ind_c in index_capoluoghi:
                if ind_r != ind_c:
                    adj[ind_r, ind_c] = 1
                
        with open(os.path.join(config['paths']['adj'],"adj_totale.pkl"), "wb") as f:
            pickle.dump(adj, f)
        
    return adj


In [29]:
def fill_dataset(df: pd, 
                 config:yaml):
    # questa procedura serve solo ad inserire i dati sulle province mancanti
    # questa procedura serve solo ad inserire i dati sulle province mancanti
    d = []
    feat = df.shape[1]
    codici = list(df.codice_provincia.unique())
    codice_reg_prov = {tuple(x) for x in df[config['dataset']['col_categorical_prov']].values.tolist()}
    codice_reg_prov = pd.DataFrame(codice_reg_prov, columns=config['dataset']['col_categorical_prov'])
    codice_reg_prov = codice_reg_prov.sort_values(by = ['codice_regione', 'codice_provincia']).reset_index(drop = True)
    
    codici = list(codice_reg_prov.codice_provincia.unique())
    for data in tqdm(df.data.unique(), desc = "filling"):
        tmp = df[df.data == data]
        if len(tmp) != len(codici):
            # individue le province mancanti
            codici_mancanti = [x for x in codici if x not in tmp.codice_provincia.unique()]
            tmp = pd.concat([tmp, codice_reg_prov[codice_reg_prov.codice_provincia.isin(codici_mancanti)]])        
            tmp.data = data
            tmp = tmp.fillna(0)
    
            if len(tmp) != len(codici):
                import pdb
                pdb.set_trace()
        d.append(tmp)
    df = pd.concat(d).sort_values(by = ['data','codice_regione', 'codice_provincia']).reset_index(drop = True)
    df = df[list(df.columns.drop(['codice_regione', 'codice_provincia']))+['codice_regione', 'codice_provincia']]
    return df
    
def get_dataset(config: yaml):
    files = ["covid", "covid_province", "covid_regioni"]
    exists = [os.path.exists(os.path.join(config['paths']['data'], f"{file}.csv")) for file in files]


    ############## Checking if all the files that I need exists #######################
    if  np.all(exists):
        tmp = pd.read_csv(os.path.join(config['paths']['data'], f"covid.csv"), index_col=0)
        tmp.data = pd.to_datetime(tmp.data, format='%Y-%m-%d')
        return tmp
        
    else:
        ########## I create take the dataset for the provinces
        if os.path.exists(os.path.join(config['paths']['data'],"covid_province.csv")):
            provincials = pd.read_csv(os.path.join(config['paths']['data'],"covid_province.csv"), index_col = 0)
            provincials['data'] = pd.to_datetime(provincials['data'], format='%Y-%m-%d')
        else:
            data = []
            for csv in tqdm(os.listdir(os.path.join(config['paths']['raw_data'],"dati-province")), desc = "provincia"):
                if csv.split(".")[-1] == "csv":
                    tmp = pd.read_csv(os.path.join(config['paths']['raw_data'],"dati-province",csv), 
                                      usecols = ['data', 'codice_regione', 'codice_provincia', 'totale_casi'])
                    data.append(tmp)
            
            data = pd.concat(data)
            data.reset_index(drop = True, inplace=True)
            data.data = data.data.apply(lambda x: x.split("T")[0])
            data.data = pd.to_datetime(data['data'], format='%Y-%m-%d')
            
            # Riduco il dataset con le variabili che servono
            # Inoltre trasformo la conta totale dei positivi per ogni regione in nuovi positivi
            provincials = data.drop_duplicates()
            provincials = provincials[-(provincials.codice_provincia>200)]
            
            # codici relative alle province che sono state male elaborate
            provincials = provincials[-provincials.codice_regione.isin([21,22])]
            provincials.to_csv(os.path.join(config['paths']['data'],"covid_province.csv"))
        
        if os.path.exists(os.path.join(config['paths']['data'], "covid_regioni.csv")):
            regions = pd.read_csv(os.path.join(config['paths']['data'],"covid_regioni.csv"), index_col=0)
            regions['data'] = pd.to_datetime(regions['data'], format='%Y-%m-%d')
        else:
            ########## I create take the dataset for the regions
            data = []
            for csv in tqdm(os.listdir(os.path.join(config['paths']['raw_data'],"dati-regioni")), desc = "regione"):
                if csv.split(".")[-1] == "csv":
                    data.append(pd.read_csv(os.path.join(config['paths']['raw_data'],"dati-regioni",csv)))
            data = pd.concat(data).drop_duplicates().reset_index(drop = True)
            
            # Trento
            data.codice_regione = data.codice_regione.replace(21, 4)
            # Bolzano
            data.codice_regione = data.codice_regione.replace(22, 4)
            # in questo modo sommo tutte le variabili delle due provincie
            # non ci sono problemi perché le variabili rimanenti sono solo numeriche 
            data = data.groupby(by = ['data', 'codice_regione']).sum().reset_index()
            
            ############# Estraggo solo le colonne che sono utili ######
            regions = data[config['dataset']['col_region']]
            regions.data = regions.data.apply(lambda x: x.split("T")[0])
            regions.data = pd.to_datetime(regions['data'], format='%Y-%m-%d')
            
            regions = regions.drop_duplicates().reset_index(drop = True)
            regions.to_csv(os.path.join(config['paths']['data'],"covid_regioni.csv"))
                    
        # Creo il dataset completo 
        if os.path.exists(os.path.join(config['paths']['data'],"covid.csv")):
            df = pd.read_csv(os.path.join(config['paths']['data'], "covid.csv"), index_col=0)
            df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d')

        else:
            df = pd.merge(provincials, regions, 
                          how = "inner",  
                          on = ['data','codice_regione']) 
            df = df.sort_values(by = ['data','codice_regione', 'codice_provincia']).reset_index(drop = True)
            
            codice_reg_prov = {tuple(x) for x in df[config['dataset']['col_categorical']].values.tolist()}
            codice_reg_prov = pd.DataFrame(codice_reg_prov, columns=config['dataset']['col_categorical'])

            ################# Cleaning the dataset ###################
            n_nodes = len(df.codice_provincia.unique())
            dates = df.data.unique()
            for data in tqdm(dates, desc = "cleaning"):
                if len(df[df.data == data].codice_provincia.unique()) != n_nodes:
                    df = df[-(df.data == data)]
            
            df.sort_values(by="data")
            df[[f"variazione_{x}" for x in config['dataset']['col_numerical']]]= df.groupby(by = 'codice_provincia').diff()[config['dataset']['col_numerical']]
            df.dropna(inplace=True)
            df.rename(columns={'variazione_totale_casi': 'nuovi_casi'}, inplace=True)
            df = df.reset_index(drop = True)
            df = df[list(df.drop(columns = config['dataset']['col_categorical']).columns)+ config['dataset']['col_categorical']]
            ################# Date variables ########################
             
            df['year'] = df.data.dt.year-df.data.dt.year.min()
            df['month'] = df.data.dt.month-df.data.dt.month.min()
            df['day'] = df.data.dt.day-df.data.dt.day.min()
            df['dayofweek'] = df.data.dt.dayofweek
            df['weekend'] = [0 if x<6 else 1 for x in df.dayofweek]

            ################# Saving the dataset #####################
            #codice_reg_prov.to_csv(os.path.join(config['paths']['data'],"codice_reg_prov.csv"))
            df.to_csv(os.path.join(config['paths']['data'],"covid.csv"))
            
    return df
    
data = get_dataset(config)
data.head(10)

cleaning: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1298/1298 [00:01<00:00, 834.57it/s]


,data,totale_casi,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,dimessi_guariti,deceduti,nuovi_casi,variazione_ricoverati_con_sintomi,...,variazione_dimessi_guariti,variazione_deceduti,variazione_isolamento_domiciliare,codice_regione,codice_provincia,year,month,day,dayofweek,weekend
0,2020-06-26,15875,307,14,321,1264,25649,4077,5.0,-9.0,...,105.0,6.0,-86.0,1,1,0,5,25,4,0
1,2020-06-26,1321,307,14,321,1264,25649,4077,0.0,-9.0,...,105.0,6.0,-86.0,1,2,0,5,25,4,0
2,2020-06-26,2787,307,14,321,1264,25649,4077,2.0,-9.0,...,105.0,6.0,-86.0,1,3,0,5,25,4,0
3,2020-06-26,2853,307,14,321,1264,25649,4077,2.0,-9.0,...,105.0,6.0,-86.0,1,4,0,5,25,4,0
4,2020-06-26,1874,307,14,321,1264,25649,4077,0.0,-9.0,...,105.0,6.0,-86.0,1,5,0,5,25,4,0
5,2020-06-26,4063,307,14,321,1264,25649,4077,0.0,-9.0,...,105.0,6.0,-86.0,1,6,0,5,25,4,0
6,2020-06-26,1045,307,14,321,1264,25649,4077,1.0,-9.0,...,105.0,6.0,-86.0,1,96,0,5,25,4,0
7,2020-06-26,1136,307,14,321,1264,25649,4077,3.0,-9.0,...,105.0,6.0,-86.0,1,103,0,5,25,4,0
8,2020-06-26,1194,2,0,2,2,1044,146,0.0,0.0,...,1.0,0.0,-1.0,2,7,0,5,25,4,0
9,2020-06-26,3886,501,47,548,11092,65323,16624,8.0,-121.0,...,492.0,16.0,-230.0,3,12,0,5,25,4,0
